In [1]:
pip list | grep netCDF4

netCDF4                   1.7.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip list | grep h5netcdf


h5netcdf                  1.6.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import os

# ------------- USER SETTINGS -------------
nc_path = "data_0.nc"
out_dir = "era5_rasters_bins"
os.makedirs(out_dir, exist_ok=True)

variables = ['t2m']                 # or any other variable(s)
time_indices = [0, 12, 24, 36, 47]  # hours you want

# If you know your temperature range (in Kelvin):
# For example, 10–30°C = 283.15–303.15 K
# Or set to None to use each image's min/max
temp_min, temp_max = 273.15, 303.15  # 10°C–30°C, change as needed
n_bins = 12  # Number of color bins

# ------------- SCRIPT --------------------
ds = xr.open_dataset(nc_path)

for var in variables:
    print(f"Plotting variable: {var}")
    arr_full = ds[var].values  # [time, lat, lon]
    for t in time_indices:
        arr = arr_full[t]
        arr = np.nan_to_num(arr, nan=0)

        # Dynamically set bounds if wanted
        if temp_min is not None and temp_max is not None:
            vmin, vmax = temp_min, temp_max
        else:
            vmin, vmax = np.percentile(arr, [2, 98])
        
        # Create bin edges for color mapping
        bounds = np.linspace(vmin, vmax, n_bins + 1)
        cmap = plt.get_cmap('plasma', n_bins)  # try 'jet', 'RdYlBu', etc.
        norm = BoundaryNorm(boundaries=bounds, ncolors=n_bins)

        # Plot discrete-color map
        img_name = f"{var}_hour{t:02d}_binned.png"
        plt.figure(figsize=(12,5))
        plt.axis('off')
        im = plt.imshow(arr, cmap=cmap, norm=norm, origin='upper')
        cbar = plt.colorbar(im, boundaries=bounds, ticks=bounds, fraction=0.025, pad=0.04)
        cbar.set_label(ds[var].attrs.get('long_name', var))
        plt.tight_layout(pad=0)
        plt.savefig(os.path.join(out_dir, img_name), bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()
        print(f"  Saved {img_name}")

print("Done! Check the", out_dir, "folder for color-binned PNG images.")


Plotting variable: t2m
  Saved t2m_hour00_binned.png
  Saved t2m_hour12_binned.png
  Saved t2m_hour24_binned.png
  Saved t2m_hour36_binned.png
  Saved t2m_hour47_binned.png
Done! Check the era5_rasters_bins folder for color-binned PNG images.


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import rasterio
from rasterio.transform import from_origin
import os

nc_path = "data_0.nc"
rasters_dir = "../era5_rasters_bins"
geotiffs_dir = "../era5_geotiffs"
os.makedirs(rasters_dir, exist_ok=True)
os.makedirs(geotiffs_dir, exist_ok=True)

# Set your temp variable and fixed bounds
temp_var = 't2m'
temp_vmin, temp_vmax = 273.15, 303.15  # 0°C to 30°C

n_bins = 12

ds = xr.open_dataset(nc_path)
lat = ds.latitude.values
lon = ds.longitude.values
transform = from_origin(lon.min(), lat.max(), abs(lon[1] - lon[0]), abs(lat[1] - lat[0]))

for var in ds.data_vars:
    arr_full = ds[var].values  # [time, lat, lon]
    for t in range(arr_full.shape[0]):
        arr = arr_full[t]
        arr = np.nan_to_num(arr, nan=0)

        # -- Color bin logic --
        if var == temp_var:
            vmin, vmax = temp_vmin, temp_vmax
        else:
            vmin, vmax = np.percentile(arr, [5, 95])

        # (protect against zero-range)
        if abs(vmax - vmin) < 1e-6:
            vmax = vmin + 1e-6

        bounds = np.linspace(vmin, vmax, n_bins + 1)
        cmap = plt.get_cmap('plasma', n_bins)import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import rasterio
from rasterio.transform import from_origin
import os

nc_path = "data_0.nc"
rasters_dir = "../era5_rasters_bins"
geotiffs_dir = "../era5_geotiffs"
os.makedirs(rasters_dir, exist_ok=True)
os.makedirs(geotiffs_dir, exist_ok=True)

# Set your temp variable and fixed bounds
temp_var = 't2m'
temp_vmin, temp_vmax = 273.15, 303.15  # 0°C to 30°C

n_bins = 12

ds = xr.open_dataset(nc_path)
lat = ds.latitude.values
lon = ds.longitude.values
transform = from_origin(lon.min(), lat.max(), abs(lon[1] - lon[0]), abs(lat[1] - lat[0]))

for var in ds.data_vars:
    arr_full = ds[var].values  # [time, lat, lon]
    for t in range(arr_full.shape[0]):
        arr = arr_full[t]
        arr = np.nan_to_num(arr, nan=0)

        # -- Color bin logic --
        if var == temp_var:
            vmin, vmax = temp_vmin, temp_vmax
        else:
            vmin, vmax = np.percentile(arr, [5, 95])

        # (protect against zero-range)
        if abs(vmax - vmin) < 1e-6:
            vmax = vmin + 1e-6

        bounds = np.linspace(vmin, vmax, n_bins + 1)
        cmap = plt.get_cmap('plasma', n_bins)
        norm = BoundaryNorm(boundaries=bounds, ncolors=n_bins)

        # PNG raster
        plt.figure(figsize=(12, 5))
        plt.axis('off')
        plt.imshow(arr, cmap=cmap, norm=norm, origin='upper')
        plt.tight_layout(pad=0)
        png_name = f"{var}_hour{t:02d}_binned.png"
        plt.savefig(os.path.join(rasters_dir, png_name), bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()

        # GeoTIFF (raw data)
        tif_name = f"{var}_hour{t:02d}.tif"
        with rasterio.open(
            os.path.join(geotiffs_dir, tif_name), 'w',
            driver='GTiff', height=arr.shape[0], width=arr.shape[1],
            count=1, dtype=arr.dtype, crs='EPSG:4326', transform=transform
        ) as dst:
            dst.write(arr, 1)
        print(f"Saved: {png_name}, {tif_name}")

print("Done!")

        norm = BoundaryNorm(boundaries=bounds, ncolors=n_bins)

        # PNG raster
        plt.figure(figsize=(12, 5))
        plt.axis('off')
        plt.imshow(arr, cmap=cmap, norm=norm, origin='upper')
        plt.tight_layout(pad=0)
        png_name = f"{var}_hour{t:02d}_binned.png"
        plt.savefig(os.path.join(rasters_dir, png_name), bbox_inches='tight', pad_inches=0, dpi=200)
        plt.close()

        # GeoTIFF (raw data)
        tif_name = f"{var}_hour{t:02d}.tif"
        with rasterio.open(
            os.path.join(geotiffs_dir, tif_name), 'w',
            driver='GTiff', height=arr.shape[0], width=arr.shape[1],
            count=1, dtype=arr.dtype, crs='EPSG:4326', transform=transform
        ) as dst:
            dst.write(arr, 1)
        print(f"Saved: {png_name}, {tif_name}")

print("Done!")


Saved: d2m_hour00_binned.png, d2m_hour00.tif
Saved: d2m_hour01_binned.png, d2m_hour01.tif
Saved: d2m_hour02_binned.png, d2m_hour02.tif
Saved: d2m_hour03_binned.png, d2m_hour03.tif
Saved: d2m_hour04_binned.png, d2m_hour04.tif
Saved: d2m_hour05_binned.png, d2m_hour05.tif
Saved: d2m_hour06_binned.png, d2m_hour06.tif
Saved: d2m_hour07_binned.png, d2m_hour07.tif
Saved: d2m_hour08_binned.png, d2m_hour08.tif
Saved: d2m_hour09_binned.png, d2m_hour09.tif
Saved: d2m_hour10_binned.png, d2m_hour10.tif
Saved: d2m_hour11_binned.png, d2m_hour11.tif
Saved: d2m_hour12_binned.png, d2m_hour12.tif
Saved: d2m_hour13_binned.png, d2m_hour13.tif
Saved: d2m_hour14_binned.png, d2m_hour14.tif
Saved: d2m_hour15_binned.png, d2m_hour15.tif
Saved: d2m_hour16_binned.png, d2m_hour16.tif
Saved: d2m_hour17_binned.png, d2m_hour17.tif
Saved: d2m_hour18_binned.png, d2m_hour18.tif
Saved: d2m_hour19_binned.png, d2m_hour19.tif
Saved: d2m_hour20_binned.png, d2m_hour20.tif
Saved: d2m_hour21_binned.png, d2m_hour21.tif
Saved: d2m

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import rasterio
from rasterio.transform import from_origin
import os, gc

# -- SETTINGS --
nc_path = "data_0.nc"
rasters_dir = "../era5_rasters_bins"
geotiffs_dir = "../era5_geotiffs"
os.makedirs(rasters_dir, exist_ok=True)
os.makedirs(geotiffs_dir, exist_ok=True)
var = 'tp'    # Change as needed

ds = xr.open_dataset(nc_path)
arr_full = ds[var].values
lat = ds.latitude.values
lon = ds.longitude.values
transform = from_origin(lon.min(), lat.max(), abs(lon[1] - lon[0]), abs(lat[1] - lat[0]))

n_bins = 12

# Use 5th-95th percentile for color limits for non-temp variables
for t in range(arr_full.shape[0]):
    arr = arr_full[t]
    arr = np.nan_to_num(arr, nan=0)
    vmin, vmax = np.percentile(arr, [5, 95])
    if abs(vmax - vmin) < 1e-6:
        vmax = vmin + 1e-6

    bounds = np.linspace(vmin, vmax, n_bins + 1)
    cmap = plt.get_cmap('plasma', n_bins)
    norm = BoundaryNorm(boundaries=bounds, ncolors=n_bins)

    # -- PNG raster --
    plt.figure(figsize=(12, 5))
    plt.axis('off')
    plt.imshow(arr, cmap=cmap, norm=norm, origin='upper')
    plt.tight_layout(pad=0)
    png_name = f"{var}_hour{t:02d}_binned.png"
    plt.savefig(os.path.join(rasters_dir, png_name), bbox_inches='tight', pad_inches=0, dpi=200)
    plt.close('all')

    # -- GeoTIFF --
    tif_name = f"{var}_hour{t:02d}.tif"
    with rasterio.open(
        os.path.join(geotiffs_dir, tif_name), 'w',
        driver='GTiff', height=arr.shape[0], width=arr.shape[1],
        count=1, dtype=arr.dtype, crs='EPSG:4326', transform=transform
    ) as dst:
        dst.write(arr, 1)
    
    print(f"Saved: {png_name}, {tif_name}")
    del arr
    gc.collect()

del arr_full, ds
gc.collect()


Saved: tp_hour00_binned.png, tp_hour00.tif
Saved: tp_hour01_binned.png, tp_hour01.tif
Saved: tp_hour02_binned.png, tp_hour02.tif
Saved: tp_hour03_binned.png, tp_hour03.tif
Saved: tp_hour04_binned.png, tp_hour04.tif
Saved: tp_hour05_binned.png, tp_hour05.tif
Saved: tp_hour06_binned.png, tp_hour06.tif
Saved: tp_hour07_binned.png, tp_hour07.tif
Saved: tp_hour08_binned.png, tp_hour08.tif
Saved: tp_hour09_binned.png, tp_hour09.tif
Saved: tp_hour10_binned.png, tp_hour10.tif
Saved: tp_hour11_binned.png, tp_hour11.tif
Saved: tp_hour12_binned.png, tp_hour12.tif
Saved: tp_hour13_binned.png, tp_hour13.tif
Saved: tp_hour14_binned.png, tp_hour14.tif
Saved: tp_hour15_binned.png, tp_hour15.tif
Saved: tp_hour16_binned.png, tp_hour16.tif
Saved: tp_hour17_binned.png, tp_hour17.tif
Saved: tp_hour18_binned.png, tp_hour18.tif
Saved: tp_hour19_binned.png, tp_hour19.tif
Saved: tp_hour20_binned.png, tp_hour20.tif
Saved: tp_hour21_binned.png, tp_hour21.tif
Saved: tp_hour22_binned.png, tp_hour22.tif
Saved: tp_h

39